# **NOTE: Copy to your drive and then run the steps one by one. (for nvidia gpu)**
- if you are using this notebook locally then remove the google colab specific package imports and code

In [ ]:
#@title Check current GPU assigned
!nvidia-smi -L
!nvidia-smi

In [ ]:
#@title 1. install / import packages (mandatory)

#install
!sudo apt install python3.10-venv
!pip install pyngrok

#import
import os
from google.colab import drive
from IPython.utils import capture
import ipywidgets as widgets
import time
import sys
import fileinput
from pyngrok import ngrok, conf
import re
from IPython.display import clear_output
from IPython.utils import capture

#functions
def inf(msg, style, wdth):
  inf = widgets.Button(description=msg, disabled=True, button_style=style, layout=widgets.Layout(min_width=wdth))
  display(inf)

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@title 2. Initiate invokeai directory (mandatory)
use_temp_storage = True #@param {type:"boolean"}
#@markdown - **recommended to use temp storage instead of gdrive for performance**
#@markdown - If not, make sure you have enough space on your gdrive as drive will be mounted.

invoke_ai_folder = "InvokeAi" #@param {type:"string"}
#@markdown - name of your existing folder or new folder to be created.

if not use_temp_storage:
  drive_path = '/content/drive'
  drive.mount(drive_path)
  invokeai_root_path = drive_path + '/MyDrive/' + invoke_ai_folder
else:
  invokeai_root_path = '/content/' + invoke_ai_folder

if not os.path.exists(invokeai_root_path):
  print("path " + invokeai_root_path + " does not exist. creating directory.")
  %mkdir $invokeai_root_path
else:
  print("existing directory found.")

%cd $invokeai_root_path

clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@title 3. initiate invokeai env and install packages inside (Mandatory - will take time)
!python -m venv .venv --prompt InvokeAI
!source .venv/bin/activate
!python -m pip install --upgrade pip
!pip install "InvokeAI[xformers]" --use-pep517 --extra-index-url https://download.pytorch.org/whl/cu118
clear_output()
inf('\u2714 Done','success', '50px')

In [ ]:
#@title 4. restart invokeai env (mandatory)
!source .venv/bin/activate

In [ ]:
#@title 5. configure invoke ai (mandatory for first time)
#@markdown - **(will take time if model download is not skipped)**
#@markdown - **(run only once if you are using gdrive, as this would already be present in drive folder)**

#@markdown 1. default install (--yes).
default_install = True #@param {type:"boolean"}
#@markdown - If yes, make sure you have enough space, as a lot of models will be downloaded if download is not skipped below.
#@markdown ---
#@markdown 2. skipping SD weights downloads (--skip-sd-weights).
skip_sd_weights_download = True #@param {type:"boolean"}
#@markdown - if yes, it will skip initial sd weights download (will save a huge amount of space)
#@markdown ---
#@markdown 3. skipping support models downloads (--skip-support-models).
skip_support_models_download = False #@param {type:"boolean"}
#@markdown - if yes, it will skip initial support models download (will save a huge amount of space)
#@markdown - dont skip unless needed or you are aware of what you are doing.
#@markdown ---
#@markdown 4. setting root dir to current dir (--root .).
root_dir = "" #@param {type:"string"}
#@markdown - root directory, (leave empty if you are not aware of it, default will be considered)
#@markdown ---


command = f"invokeai-configure {'--yes' if default_install else ''} {'--skip-sd-weights' if skip_sd_weights_download else ''} {'--skip-support-models' if skip_support_models_download else ''} --root {'.' if root_dir=='' else root_dir}"
# !invokeai-configure --yes --root .

try:
  !{command}
  clear_output()
  inf('\u2714 Done','success', '50px')
except Exception as e:
  print(e)
  inf('\u2714 Failed','danger', '50px')

In [ ]:
#@title  6. Start tunnel

Ngrok_token = "" #@param {type:"string"}
#@markdown - insert your ngrok account auth token to be able to access invoke ai on a publicly accessible link.
#@markdown - connects on local port: 9090 - tunnel - public
#@markdown - link will be shown in output

if Ngrok_token!="":
  ngrok.kill()
  url=ngrok.connect(9090, pyngrok_config=conf.PyngrokConfig(auth_token=Ngrok_token) , bind_tls=True).public_url
  print("ai will start soon at following link: ",url)

In [ ]:
#@title 7. Start invokeai (mandatory)
!invokeai-web